<a href="https://colab.research.google.com/github/shr-ey-a/Leetcode/blob/main/DermaSCAN_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras matplotlib pillow streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import zipfile

# Upload the dataset ZIP file
uploaded = files.upload()

# Extract the dataset
for file_name in uploaded.keys():
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall("/content/skin_disease_dataset")


Saving DermaScan.zip to DermaScan (1).zip


In [ ]:
import os

# Correct dataset path
base_path = "/content/skin_disease_dataset/skin-disease-datasaet"

print("Folders inside dataset:", os.listdir(base_path))


Folders inside dataset: ['test_set', 'train_set']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image size (standard for CNNs)
img_size = (128, 128)
batch_size = 32

# Create generators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    base_path + "/train_set",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    base_path + "/test_set",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 924 images belonging to 8 classes.
Found 233 images belonging to 8 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # helps prevent overfitting
    layers.Dense(8, activation='softmax')  # 8 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Show model summary
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,672 (12.61 MB)

 Trainable params: 3,305,672 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,   # Start with 10, you can increase later
    validation_data=test_generator
)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.1382 - loss: 2.2796 - val_accuracy: 0.1545 - val_loss: 2.0160
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.1946 - loss: 1.9836 - val_accuracy: 0.3777 - val_loss: 1.7588
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.3760 - loss: 1.7193 - val_accuracy: 0.4335 - val_loss: 1.5893
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4772 - loss: 1.4938 - val_accuracy: 0.5107 - val_loss: 1.4193
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.5384 - loss: 1.3347 - val_accuracy: 0.5451 - val_loss: 1.2611
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.6383 - loss: 1.0213 - val_accuracy: 0.5622 - val_loss: 1.2059
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.6766 - loss: 0.9198 - val_accuracy: 0.6223 - val_loss: 1.0596
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.7265 - loss: 0.7889 - val_accuracy: 0.6438 - val_loss: 1.0847
Epo

In [ ]:
# Evaluate on test set
loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc*100:.2f}%")


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.6994 - loss: 1.0085
Test Accuracy: 70.82%


In [ ]:
import os

# test folder path
test_path = base_path + "/test_set"

# list available class folders
print("Available classes:", os.listdir(test_path))



Available classes: ['FU-nail-fungus', 'VI-chickenpox', 'BA- cellulitis', 'VI-shingles', 'FU-ringworm', 'BA-impetigo', 'FU-athlete-foot', 'PA-cutaneous-larva-migrans']


In [ ]:
print("Files in FU-ringworm:", os.listdir(test_path + "/FU-ringworm")[:5])


Files in FU-ringworm: ['42_FU-ringworm (36).jpg', '107_FU-ringworm (16).jpg', '40_FU-ringworm (17).jpg', '50_FU-ringworm (36).jpg', '16_FU-ringworm (59).jpg']


In [ ]:
import random
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Step 7 - Pick a random image from test_set
# -------------------------------------------

# list all class folders
class_folders = os.listdir(test_path)

# randomly select one class
true_class = random.choice(class_folders)

# list all images inside that class folder
image_files = os.listdir(os.path.join(test_path, true_class))

# randomly select one image file
chosen_image = random.choice(image_files)

# get full path of chosen image
img_path = os.path.join(test_path, true_class, chosen_image)

print("Testing Image Path:", img_path)
print("True Class (from folder):", true_class)

# load and preprocess the image
img = image.load_img(img_path, target_size=img_size)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# predict using model
prediction = model.predict(img_array)
class_idx = np.argmax(prediction)
# predict using model
prediction = model.predict(img_array)
class_idx = np.argmax(prediction)

# class labels taken from the test folder names
class_labels = sorted(os.listdir(test_path))
print("Class Labels:", class_labels)

print("Predicted Class:", class_labels[class_idx])




Testing Image Path: /content/skin_disease_dataset/skin-disease-datasaet/test_set/PA-cutaneous-larva-migrans/72_PA-cutaneous-larva-migrans (19).jpg
True Class (from folder): PA-cutaneous-larva-migrans
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Class Labels: ['BA- cellulitis', 'BA-impetigo', 'FU-athlete-foot', 'FU-nail-fungus', 'FU-ringworm', 'PA-cutaneous-larva-migrans', 'VI-chickenpox', 'VI-shingles']
Predicted Class: PA-cutaneous-larva-migrans


In [ ]:
# get true class from the folder name (path of the test image)
true_class = os.path.basename(os.path.dirname(img_path))   # <-- use img_path

# predicted class
predicted_class = class_labels[class_idx]

print("True Class:     ", true_class)
print("Predicted Class:", predicted_class)

# check match
if true_class == predicted_class:
    print("✅ Correct Prediction!")
else:
    print("❌ Wrong Prediction.")


True Class:      PA-cutaneous-larva-migrans
Predicted Class: PA-cutaneous-larva-migrans
✅ Correct Prediction!


In [ ]:
# A1) Save your trained model
model.save("dermascan_ai.h5")
print("Saved: dermascan_ai.h5")

# A2) Save class labels in the correct output order
import json

# train_generator.class_indices is like {'BA-impetigo':0, 'FU-ringworm':1, ...}
idx_to_class = {v: k for k, v in train_generator.class_indices.items()}
labels_in_output_order = [idx_to_class[i] for i in range(len(idx_to_class))]

with open("labels.json", "w") as f:
    json.dump(labels_in_output_order, f)

print("Saved: labels.json")
print("Labels (in model output order):", labels_in_output_order)


Saved: dermascan_ai.h5
Saved: labels.json
Labels (in model output order): ['BA- cellulitis', 'BA-impetigo', 'FU-athlete-foot', 'FU-nail-fungus', 'FU-ringworm', 'PA-cutaneous-larva-migrans', 'VI-chickenpox', 'VI-shingles']


In [ ]:
!pip install -q gradio



In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import json

# C1) Load model and labels
MODEL_PATH = "dermascan_ai.h5"
LABELS_PATH = "labels.json"
IMG_SIZE = 128  # keep this SAME as what you used during training

model = tf.keras.models.load_model(MODEL_PATH)
with open(LABELS_PATH, "r") as f:
    CLASS_LABELS = json.load(f)

# C2) Preprocess + predict
def predict_skin(img: Image.Image):
    # ensure RGB
    img = img.convert("RGB").resize((IMG_SIZE, IMG_SIZE))
    x = np.array(img, dtype=np.float32) / 258.0
    x = np.expand_dims(x, axis=0)  # shape: (1, 128, 128, 3)

    preds = model.predict(x)  # shape: (1, num_classes)
    probs = preds[0].tolist()

    # return dict {label: probability}
    return {CLASS_LABELS[i]: float(probs[i]) for i in range(len(CLASS_LABELS))}

# C3) Build Gradio UI
demo = gr.Interface(
    fn=predict_skin,
    inputs=gr.Image(type="pil", label="Upload a Clear Skin Image"),
    outputs=gr.Label(num_top_classes=3, label="TOP PREDICTIONS"),
    title="DermaScan AI",
    description="Prototype: AI-Powered Skin Disease Detection",
    allow_flagging="never"
)

# C4) Launch — share=True gives you a public URL for the expo
demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0bd148db8461b72141.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os, random

# create a few example paths from your test_set
test_path = "/content/skin_disease_dataset/skin-disease-datasaet/test_set"
examples = []
for cls in sorted(os.listdir(test_path))[:4]:  # pick first 4 classes
    cls_dir = os.path.join(test_path, cls)
    files = [f for f in os.listdir(cls_dir) if f.lower().endswith((".jpg",".jpeg",".png"))]
    if files:
        examples.append(os.path.join(cls_dir, random.choice(files)))

demo = gr.Interface(
    fn=predict_skin,
    inputs=gr.Image(type="pil", label="Upload a Clear Skin Image"),
    outputs=gr.Label(num_top_classes=3, label="Top Predictions"),
    title="DermaScan AI",
    description="Prototype:AI-Assisted Skin Condition Classifier.",
    examples=examples,
    allow_flagging="never"
)
demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://658e4cdf8cc3f9f477.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 1) Save the model file (change the filename if you prefer)
model.save("skin_disease_model.h5")
print("Saved: skin_disease_model.h5")

# 2) Save labels in model output order (important)
import json
# If you used a generator like train_generator earlier:
idx_to_class = {v: k for k, v in train_generator.class_indices.items()}
labels_in_order = [idx_to_class[i] for i in range(len(idx_to_class))]
with open("labels.json", "w") as f:
    json.dump(labels_in_order, f)
print("Saved: labels.json")
print("Labels:", labels_in_order)

# 3) Show file sizes so you know how big to download
!ls -lh skin_disease_model.h5 labels.json


In [ ]:
from google.colab import files
files.download('skin_disease_model.h5')
files.download('labels.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>